# Destiny 2 Weapon Roll Extraction
This notebook interactively extracts and classifies all possible and equipped Destiny 2 weapon perks for your account, similar to D2Checklist or DIM.

In [1]:
# --- Imports and Environment Setup ---
import os
import sys
import json
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from supabase import create_client

# Adjust sys.path to include the project root
SCRIPT_DIR = os.path.dirname(os.path.realpath("."))
PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, os.pardir))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Load environment variables
load_dotenv()
BUNGIE_API_KEY = os.getenv("BUNGIE_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BUNGIE_ACCESS_TOKEN = os.getenv("BUNGIE_ACCESS_TOKEN")

# Logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [2]:
import os
import sys

# Set this to the absolute path of your project root (where web_app/ lives)
PROJECT_ROOT = "/Users/davehiltbrand/Documents/destiny2_catalysts"
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
print("sys.path:", sys.path)

sys.path: ['/Users/davehiltbrand/Documents/destiny2_catalysts', '/Users', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/davehiltbrand/Documents/destiny2_catalysts/venv/lib/python3.11/site-packages']


In [3]:
TOKEN_PATH = os.path.join(PROJECT_ROOT, "token.json")

with open(TOKEN_PATH, "r") as f:
    token_data = json.load(f)
print(token_data)

{'access_token': 'CKKkBxKGAgAgShSOaJ0w2p3G0z381ks9v1P4dTRAkmyXhHQWywbCm4LgAAAArf35Q8kA5xI/ZKgL818fmywThP7p9eEq0uDc495Yvokw7tRvQXxAVLa5zDo9ul90PbNZCPQXFL1he0Dp+thJECs8ONCuJ5CVCjb8C7JQu7iH3hZvGYEoXxbAaiRjnqoP6qNXRlIE8EfBmt/C3p/4fk6vMi55ucfqPwhtAT/4r7VuEw8mMrLgPNWZRT0pv75v94JUOUsZ/2n+2LWJj2taqFhemmF6CP0A4RYTuHsakWg/oFDdJBwlvY9BzdEi9GBvrEP6ZDBJuJPjHtk7orVcNM7jcSvvMAuWVmphTRGdgYg=', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'CKKkBxKGAgAgQBay1hDzN8nV3dXgQHVye8J4Pm93L3M1+OxQo/hTnBTgAAAAXU5g1RJVTx2nVFzblq0/TBL16i5YRZ3Afyczu8suMU2+ExOXHRpYF4da+W/BJ7p1sv93yJklzHSkEr9NqZwtYJPkm65VpE4CnTX7fEMfdDsusqy9shtbn8SoEc8RfpLTvmgO1UsLaBl4kQXYElS9utT/U3zMDS4GMGxjoqIqKMuZRW+LtrZilckV72W3n+pArzgClDY2uAB7UG52UwKDl4AIme27b9RLGzG8KgbmGuDWxPxPsoqzPBvQHwOsHDkrURn0ykBAzeSdkX5Tq98BLctthQRzfdHvx5GvyetMn7o=', 'refresh_expires_in': 7776000, 'membership_id': '29565467', 'received_at': '2025-05-13T00:59:55.132846'}


In [4]:
import os

# Set this to your project root where token.json lives
PROJECT_ROOT = "/Users/davehiltbrand/Documents/destiny2_catalysts"
os.chdir(PROJECT_ROOT)
print("Current working directory:", os.getcwd())
print("token.json exists:", os.path.exists("token.json"))

Current working directory: /Users/davehiltbrand/Documents/destiny2_catalysts
token.json exists: True


In [5]:
# --- Supabase and Bungie API Setup ---
from web_app.backend.weapon_api import WeaponAPI
from web_app.backend.manifest import SupabaseManifestService
from web_app.backend.bungie_oauth import OAuthManager
# Add the parent directory to sys.path to find web_app module


sb_client = create_client(SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY)
manifest_service = SupabaseManifestService(sb_client=sb_client)
oauth_manager = OAuthManager()
weapon_api = WeaponAPI(oauth_manager=oauth_manager, manifest_service=manifest_service)


2025-05-13 09:55:45,341 - INFO - [DEBUG] Attempting to load token data from file...
2025-05-13 09:55:45,341 - INFO - [DEBUG] Raw data loaded from token.json: {'access_token': 'CKKkBxKGAgAgShSOaJ0w2p3G0z381ks9v1P4dTRAkmyXhHQWywbCm4LgAAAArf35Q8kA5xI/ZKgL818fmywThP7p9eEq0uDc495Yvokw7tRvQXxAVLa5zDo9ul90PbNZCPQXFL1he0Dp+thJECs8ONCuJ5CVCjb8C7JQu7iH3hZvGYEoXxbAaiRjnqoP6qNXRlIE8EfBmt/C3p/4fk6vMi55ucfqPwhtAT/4r7VuEw8mMrLgPNWZRT0pv75v94JUOUsZ/2n+2LWJj2taqFhemmF6CP0A4RYTuHsakWg/oFDdJBwlvY9BzdEi9GBvrEP6ZDBJuJPjHtk7orVcNM7jcSvvMAuWVmphTRGdgYg=', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'CKKkBxKGAgAgQBay1hDzN8nV3dXgQHVye8J4Pm93L3M1+OxQo/hTnBTgAAAAXU5g1RJVTx2nVFzblq0/TBL16i5YRZ3Afyczu8suMU2+ExOXHRpYF4da+W/BJ7p1sv93yJklzHSkEr9NqZwtYJPkm65VpE4CnTX7fEMfdDsusqy9shtbn8SoEc8RfpLTvmgO1UsLaBl4kQXYElS9utT/U3zMDS4GMGxjoqIqKMuZRW+LtrZilckV72W3n+pArzgClDY2uAB7UG52UwKDl4AIme27b9RLGzG8KgbmGuDWxPxPsoqzPBvQHwOsHDkrURn0ykBAzeSdkX5Tq98BLctthQRzfdHvx5GvyetMn7o=', 'refresh_expires_in': 7776000, 'memb

In [6]:
# --- Fetch Profile and Prepare Data ---
membership_info = weapon_api.get_membership_info()
if not membership_info:
    raise Exception("Could not fetch membership info from Bungie API.")
membership_type = membership_info["type"]
destiny_membership_id = membership_info["id"]

components = [102, 201, 205, 310]
profile_response_data = weapon_api.get_profile_response(
    membership_type=membership_type,
    destiny_membership_id=destiny_membership_id,
    components=components
)

if not profile_response_data:
    raise Exception("Failed to get profile response data from WeaponAPI.")
profile_data = profile_response_data["Response"]
character_equip = profile_data["characterEquipment"]["data"]
character_inventories = profile_data["characterInventories"]["data"]
profile_inventory = profile_data["profileInventory"]["data"]
# item_sockets = profile_data["itemComponents"]["sockets"]["data"]
# item_instances = profile_data["itemComponents"]["instances"]["data"]
# profile_plugsets = profile_data.get("profilePlugSets", {}).get("data", {}).get("plugs", {})
# character_plugsets = profile_data.get("characterPlugSets", {}).get("data", {})
reusable_plugs = profile_data['itemComponents']['reusablePlugs']['data']


2025-05-13 09:55:46,299 - INFO - Token expired or nearing expiry, attempting refresh.
2025-05-13 09:55:46,300 - INFO - Refreshing token using internally stored refresh token.
2025-05-13 09:55:46,610 - INFO - Token refreshed successfully.
2025-05-13 09:55:46,611 - INFO - [DEBUG] Saving token data dictionary: {'access_token': 'CMukBxKGAgAgp76wUelqTzBev5CPfGabIsJAbqwwzcTru9oIKwtXBUngAAAAhdckZUvHPZg+N7ykbfc4NFa7Jxbq1qUtu64eAD/8VxshFbk+HEDNPgDtMIvhZ1FDaPwUHRP0dp/Rz1b57+gEIQwHJmq1B+Vfu1jbhnhM3RDZmdD6EcA2/hGtsIWVD3FmBp0mhLDBqXvzBEI3w9JoSETMI3/b3QLvZjjYszGK4HR+avebWgQdLLrlrAGg7Z12ll1l1TjlMsIaBOFv/BsLdR8V2S3y8SVpIwjnzkrdHQKhxKcZEOt2NUXGfzbG73Daaiqupz8X3Nn5cznrwwWEqNH7E/7x8zfxf1LYAdaXVqk=', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'CMukBxKGAgAg1+LcyAmLTHV/jXinYknqcaaGdGjl1m+ggRGS6viKDr7gAAAAyJVna1NUEqS4MoKwOYtNk0CSaWIOTCFLRv7cZeTbGhZnZgwOwb003YANBeuyCsVKLf0vZ32cFA3AVOnjWpxp0KX3PQQmYQBaDczNbYG/7sXJ2KUxDIdcrMWVXL5xklUohIaQAReTO2MZwLZx9FCLc+OW+yDDy0qM8MNC3p82Ii7Cszcu75yGyEm2fdLg

In [7]:
profile_data['itemComponents'].keys()

dict_keys(['reusablePlugs'])

In [8]:
# --- Flatten all items and process up to 10 weapons ---
from itertools import chain
all_items = list(chain(
    *(v.get("items", []) for v in character_equip.values()),
    *(v.get("items", []) for v in character_inventories.values()),
    profile_inventory.get("items", []),
))



In [9]:
import random
random.shuffle(all_items)
for item in all_items:
    item_hash = item.get('itemHash')
    item_def = manifest_service.get_definition('DestinyInventoryItemDefinition', item_hash)
    if item_def.get('itemType') == 3:  # not a weapon
        break

instance_id = item.get('itemInstanceId')
print(f'Processing {item_def["displayProperties"]["name"]} ({instance_id})')

2025-05-13 09:55:53,664 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=json_data&hash=eq.2842471112 "HTTP/2 200 OK"
2025-05-13 09:55:53,840 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=json_data&hash=eq.615393850 "HTTP/2 200 OK"
2025-05-13 09:55:54,014 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=json_data&hash=eq.1866778462 "HTTP/2 200 OK"


Processing The Hothead (Adept) (6917529803687008168)


In [10]:
reusable_plugs.get(instance_id, {})

{'plugs': {'1': [{'plugItemHash': 981914802,
    'canInsert': True,
    'enabled': True},
   {'plugItemHash': 3525010810, 'canInsert': True, 'enabled': True}],
  '2': [{'plugItemHash': 1996142143, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 3492396210, 'canInsert': True, 'enabled': True}],
  '3': [{'plugItemHash': 2869569095, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 951095735, 'canInsert': True, 'enabled': True}],
  '4': [{'plugItemHash': 3194351027, 'canInsert': True, 'enabled': True}],
  '5': [{'plugItemHash': 4248210736, 'canInsert': True, 'enabled': True}],
  '6': [{'plugItemHash': 229003538, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 2299766748, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 4278960718, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 634781242, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 1525622117, 'canInsert': True, 'enabled': True},
   {'plugItemHash': 1710791394, 'canInsert': True, 'ena

In [11]:
reusable_plugs_data = profile_response_data.get("Response", {}).get("itemComponents", {}).get("reusablePlugs", {}).get("data", {})
print(f"[DEBUG] reusable_plugs_data keys: {list(reusable_plugs_data.keys())} (count: {len(reusable_plugs_data)})")
instance_reusable_plugs = reusable_plugs_data.get(instance_id, {}).get('plugs', {})
instance_reusable_plugs

[DEBUG] reusable_plugs_data keys: ['6917530110443598914', '6917529603870481739', '6917529871647767419', '6917529819069510174', '6917529617647242171', '6917529662540121834', '6917529801112623330', '6917529806490869337', '6917529835255814812', '6917529990397295727', '6917529867795149587', '6917529475949466141', '6917529619688663567', '6917529934675429376', '6917529934675429899', '6917529933980104631', '6917529930190442138', '6917529931288722132', '6917529931589518556', '6917529937560018316', '6917529935705442395', '6917529607550277784', '6917529686490795757', '6917529932278441412', '6917529930120527063', '6917529930113795729', '6917529930543808058', '6917529930106867061', '6917529930120529872', '6917529878388215364', '6917529990397296072', '6917529797961593979', '6917529607550275197', '6917530111462386496', '6917529941047491787', '6917530110796452069', '6917530111444227557', '6917530111464930061', '6917530110803967924', '6917530111464931450', '6917530110553875057', '6917530111460872952',

{'1': [{'plugItemHash': 981914802, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 3525010810, 'canInsert': True, 'enabled': True}],
 '2': [{'plugItemHash': 1996142143, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 3492396210, 'canInsert': True, 'enabled': True}],
 '3': [{'plugItemHash': 2869569095, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 951095735, 'canInsert': True, 'enabled': True}],
 '4': [{'plugItemHash': 3194351027, 'canInsert': True, 'enabled': True}],
 '5': [{'plugItemHash': 4248210736, 'canInsert': True, 'enabled': True}],
 '6': [{'plugItemHash': 229003538, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 2299766748, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 4278960718, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 634781242, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 1525622117, 'canInsert': True, 'enabled': True},
  {'plugItemHash': 1710791394, 'canInsert': True, 'enabled': True}],
 '8': [{'plugIte

In [12]:
# --- Debug print for reusable plugs ---
socket_plug_hashes = {}
if instance_reusable_plugs:
    for plug in instance_reusable_plugs:
        # print(plug)
        plug_hashes = instance_reusable_plugs.get(plug, [])
        # print(plug_hashes)
        # iplug = instance_reusable_plugs.get(plug, [])
        plug_item_hashes = [plug_hash['plugItemHash'] for plug_hash in plug_hashes]
        # print(plug_item_hashes)
        socket_plug_hashes[plug] = plug_item_hashes
        # print(iplug.get(plug, []))
        # if plugs:
        #     num_sockets_with_reusable += 1
        #     num_total_reusable_plugs += len(plugs)

In [13]:
all_plug_hashes = set()
for plug_list in socket_plug_hashes.values():
    all_plug_hashes.update(plug_list)

In [14]:
# 2. Batch fetch all plug definitions
plug_definitions = manifest_service.get_definitions_batch(
    'DestinyInventoryItemDefinition',
    list(all_plug_hashes)
)

2025-05-13 09:56:53,731 - INFO - Starting batch fetch for 16 definitions from destinyinventoryitemdefinition in 1 chunk(s).


2025-05-13 09:56:54,291 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=hash%2Cjson_data&hash=in.%281710791394%2C1525622117%2C2869569095%2C744217850%2C4278960718%2C4248210736%2C1281216113%2C3492396210%2C981914802%2C3194351027%2C229003538%2C634781242%2C951095735%2C3525010810%2C2299766748%2C1996142143%29 "HTTP/2 200 OK"
2025-05-13 09:56:54,325 - INFO - Batch fetch complete for destinyinventoryitemdefinition. Total definitions fetched: 16 out of 16 requested.


In [15]:
socket_plug_defs = {}
for socket_index, plug_hashes in socket_plug_hashes.items():
    socket_plug_defs[socket_index] = [
        plug_definitions.get(plug_hash) for plug_hash in plug_hashes if plug_definitions.get(plug_hash)
    ]

In [16]:
# Assuming socket_plug_defs is already built as described earlier
for socket_index, plug_defs in socket_plug_defs.items():
    plug_names = [plug_def['displayProperties']['name'] for plug_def in plug_defs if plug_def]
    print(f"Socket {socket_index}: {plug_names}")

Socket 1: ['Hard Launch', 'Quick Launch']
Socket 2: ['Black Powder', 'Implosion Rounds']
Socket 3: ['Field Prep', 'Impulse Amplifier']
Socket 4: ['Explosive Light']
Socket 5: ['Default Shader']
Socket 6: ['Adept Blast Radius', 'Adept Projectile Speed', 'Adept Handling', 'Adept Reload', 'Adept Counterbalance', 'Adept Targeting']
Socket 8: ['Stunning Recovery', "Vanguard's Vindication"]


In [21]:
# Define your PCI/category mappings
PCI_COL1 = {"barrels", "tubes", "bowstrings", "blades", "hafts", "scopes"}
PCI_COL2 = {"magazines", "batteries", "guards", "arrows"}
TRAIT_PCI = {"frames", "grips", "traits"}
ORIGIN_PCI = {"origin"}
MASTERWORK_PCI = {"masterworks"}

def get_plug_category(plug_def):
    pci = plug_def.get('plug', {}).get('plugCategoryIdentifier', '').lower()
    name = plug_def.get('displayProperties', {}).get('name', '')
    item_type_display_name = plug_def.get('itemTypeDisplayName', '').lower()
    if any(key in pci for key in PCI_COL1):
        return "col1_barrel"
    elif any(key in pci for key in PCI_COL2):
        return "col2_magazine"
    elif any(key in pci for key in TRAIT_PCI) or plug_def.get('itemTypeDisplayName') in ("Trait", "Enhanced Trait", "Grip"):
        return "trait"
    elif any(key in pci for key in ORIGIN_PCI) or plug_def.get('itemTypeDisplayName') == "Origin Trait":
        return "origin_trait"
    elif 'masterworks' in pci and name.startswith('Masterworked:'):
        return "masterwork"
    elif "shader" in pci:
        return "shader"
    elif "weapon.mod_guns" in pci or "weapon mod" in item_type_display_name:
        return "weapon_mod"
    else:
        return "other"
    
# 1. Identify all trait sockets (by your PCI logic or however you already do it)
trait_socket_indexes = []
for socket_index, plug_defs in socket_plug_defs.items():
    # If any plug in this socket is a trait, consider this a trait socket
    if any(get_plug_category(plug_def) == "trait" for plug_def in plug_defs if plug_def):
        trait_socket_indexes.append(socket_index)

# 2. Sort trait sockets for consistent ordering
trait_socket_indexes = sorted(trait_socket_indexes)

# 3. Print with col3_trait1/col4_trait2 labels
for socket_index, plug_defs in socket_plug_defs.items():
    for plug_def in plug_defs:
        if not plug_def:
            continue
        name = plug_def['displayProperties']['name']
        category = get_plug_category(plug_def)
        item_hash = plug_def.get('hash')
        # Determine trait column label if this is a trait
        if category == "trait":
            if socket_index == trait_socket_indexes[0]:
                trait_label = "col3_trait1"
            elif len(trait_socket_indexes) > 1 and socket_index == trait_socket_indexes[1]:
                trait_label = "col4_trait2"
            else:
                trait_label = "trait"
            print(f"Socket {socket_index}: {name} (hash: {item_hash}) -> {trait_label}")
        else:
            print(f"Socket {socket_index}: {name} (hash: {item_hash}) -> {category}")

Socket 1: Hard Launch (hash: 981914802) -> col1_barrel
Socket 1: Quick Launch (hash: 3525010810) -> col1_barrel
Socket 2: Black Powder (hash: 1996142143) -> col2_magazine
Socket 2: Implosion Rounds (hash: 3492396210) -> col2_magazine
Socket 3: Field Prep (hash: 2869569095) -> col3_trait1
Socket 3: Impulse Amplifier (hash: 951095735) -> col3_trait1
Socket 4: Explosive Light (hash: 3194351027) -> col4_trait2
Socket 5: Default Shader (hash: 4248210736) -> shader
Socket 6: Adept Blast Radius (hash: 229003538) -> weapon_mod
Socket 6: Adept Projectile Speed (hash: 2299766748) -> weapon_mod
Socket 6: Adept Handling (hash: 4278960718) -> weapon_mod
Socket 6: Adept Reload (hash: 634781242) -> weapon_mod
Socket 6: Adept Counterbalance (hash: 1525622117) -> weapon_mod
Socket 6: Adept Targeting (hash: 1710791394) -> weapon_mod
Socket 8: Stunning Recovery (hash: 1281216113) -> origin_trait
Socket 8: Vanguard's Vindication (hash: 744217850) -> origin_trait
